<a href="https://colab.research.google.com/github/pa1tech/anveshak/blob/master/Universal_Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importing necesssary modules
from pylab import *
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix

#Universal encoding library
import tensorflow as tf
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

#Data location prefix
from google.colab import drive
drive.mount('/content/drive')
prefix = '/content/drive/My Drive/EE5180/Project/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [72]:
data = pd.read_csv(prefix + "questions.csv")
data.sort_values(by=['is_duplicate'], inplace=True)
data = data.drop(['id','qid1','qid2'],axis=1)
data = data.reset_index(drop=True)

(unique, counts) = np.unique(data.is_duplicate, return_counts=True)
n = max(counts)
dataArray = column_stack(( data.question1[n-125000:n+125000], data.question2[n-125000:n+125000], data.is_duplicate[n-125000:n+125000]))

data.head(-5)

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is Isolated earthing on ships?,What are some good career directions for a com...,0
2,Nike Secret Tournament (Scorpion KO): was it s...,Which is better: Nike or Under Armour?,0
3,What is the eligibility criteria to sit in cam...,What is the eligibility criteria to sit in cam...,0
4,What are some good ways to introduce our scave...,What is it like to live in a sorority house?,0
...,...,...,...
404341,In what way do you think the reservation syste...,Is there any way to abolish caste based reserv...,1
404342,Why is Australia not considered an Island?,Why is Australia considered a continent and no...,1
404343,What best new products or inventions don’t mos...,What are the best new products tools or invent...,1
404344,Which is the best way to prepare for SSC CGL a...,How do I crack the SSC CGL exam without coaching?,1


In [74]:
train_data, test_data = train_test_split(dataArray, test_size= 0.2)
(unique, counts) = np.unique(train_data[:,2], return_counts=True)
n = np.min(counts)
print(counts)

(unique, counts) = np.unique(test_data[:,2], return_counts=True)
n = np.min(counts)
print(counts)


[ 99930 100070]
[25070 24930]


# Universal encoding

In [75]:
#Preprocessiong
vectorData_train = []; labels_train = []
for w in train_data:
  try:
    embeddings = embed(w[:2])
    emdeddings = concatenate( (array(embeddings[0]),array(embeddings[1])) ) #Cocatenating embeddings of two sentences
    vectorData_train.append(emdeddings) 
    labels_train.append(w[2])
  except :
    pass
vectorData_train = array(vectorData_train)
labels_train = array(labels_train)
labels_train = labels_train*2 - ones(len(labels_train))

vectorData_test = []; labels_test = []
for w in test_data:
  try:
    embeddings = embed(w[:2])
    emdeddings = concatenate((array(embeddings[0]),array(embeddings[1])))
    vectorData_test.append(emdeddings) 
    labels_test.append(w[2])
  except:
    pass
vectorData_test = array(vectorData_test)
labels_test = array(labels_test)
labels_test = labels_test*2 - ones(len(labels_test))

In [76]:
print( 'Size of train data:'+ str(len(vectorData_train)))
print( 'Size of test data:'+ str(len(vectorData_test)) )

Size of train data:199999
Size of test data:50000


## Logistic Regression

In [84]:
#Logistic Regression
lreg = LogisticRegression(max_iter=1000).fit(vectorData_train, labels_train)
train = lreg.score(vectorData_train, labels_train)
test = lreg.score(vectorData_test, labels_test)

print( 'Train accuracy: '+ str(100*train) + '%')
print( 'Test accuracy: '+ str(100*test) + '%')

Train accuracy: 70.56335281676408%
Test accuracy: 70.054%


In [85]:
#Saving model
Pkl_Filename = prefix+"UnivEnc_LogReg_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(reg, file)

## Random Forest

In [78]:
#Random forest
reg = RandomForestClassifier(max_depth = 20,random_state=0).fit(vectorData_train, labels_train)
train = reg.score(vectorData_train, labels_train)
test = reg.score(vectorData_test, labels_test)

print( 'Train accuracy: '+ str(100*train) + '%')
print( 'Test accuracy: '+ str(100*test) + '%')

Train accuracy: 99.43949719748598%
Test accuracy: 77.96600000000001%


In [80]:
#Saving Model
Pkl_Filename = prefix+"UnivEnc_RandFor_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(reg, file)

In [88]:
#Confusion matrix
labels_test_pred = reg.predict(vectorData_test)
cm_rf = confusion_matrix(labels_test, labels_test_pred)
print("Accuracy for Duplicates: " + str(100*cm_rf[0,0]/(cm_rf[0,0]+cm_rf[0,1])) + "%")
print("Accuracy for not Duplicates: " + str(100*cm_rf[1,1]/(cm_rf[1,0]+cm_rf[1,1])) + "%")

Accuracy for Duplicates: 82.29756681292382%
Accuracy for not Duplicates: 73.61010830324909%


In [83]:
#Saving test data results
df = pd.DataFrame(test_data[:,0], columns=["Q1"])
df["Q2"] = test_data[:,1]
df["Vector"] = vectorData_test.tolist()
df["actual"] = labels_test
df["pred"] = labels_test_pred

df.to_csv(prefix + 'UnivEnc_RandForest.csv')
df.head(-5)

,Q1,Q2,Vector,actual,pred
0,Can I earn money online?,What is the easiest way to make a little money...,"[-0.06412064284086227, -0.07820030301809311, -...",1.0,1.0
1,Do you think the passing of the GST bill propo...,How is GST beneficial for a common man?,"[0.02641914412379265, -0.07629140466451645, 0....",1.0,1.0
2,Which authors book do I buy to learn about bod...,Which is the best interesting book to know abo...,"[0.05446075648069382, -0.0208369679749012, 0.0...",1.0,-1.0
3,What is bitmasking? What kind of problems can ...,How do I use IT to solve problems of corruption?,"[0.007975328713655472, -0.07790495455265045, -...",-1.0,-1.0
4,Can women who are bisexual realistically marry...,What gender of people do trans women marry?,"[0.02759035862982273, -0.06169082969427109, 0....",-1.0,-1.0
...,...,...,...,...,...
49990,How does CodeIgniter compare to Laravel?,How does CodeIgniter compare to CakePHP?,"[-0.005074884742498398, -0.051356393843889236,...",-1.0,-1.0
49991,Where can I get an expert painter in Brisbane?,Where can I found expert teams of interior res...,"[-0.020382607355713844, 0.01232385914772749, 0...",-1.0,-1.0
49992,Who needs dog training?,How are assistance dogs trained?,"[0.05044051632285118, -0.06690870225429535, -0...",-1.0,-1.0
49993,What are the Alto California Missions?,Is it okay to drink Palo Alto tap water?,"[-0.06566717475652695, -0.06124785915017128, 0...",-1.0,-1.0
